In [2]:
#Import
import os
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA # this will make question and answering
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone


# I am going to import my PDF so i need to import DirectoryLoader
from langchain.document_loaders import PyPDFLoader,DirectoryLoader

# Corpus to Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


# we are using quant size model so we have to import ctransformer
from langchain.llms import CTransformers

# Actually i am getting erro when i use CTransformer so i am going to use OpenAI=embeddiigng

c:\Users\arung\anaconda3\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
key=os.getenv("PINECONE_API_KEY")

In [4]:
# Dont use mine because i will delete it after completion of my project
PINECONE_API_KEY="1a0b20f7-7500-40fe-823a-dbdb343f593f"
PINECONE_API_ENV="gcp-starter"

# Load PDF

In [4]:
# load my pdf using Directory Loader
# We can load multiple pdf file also
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [5]:
extracted_data=load_pdf("Data/") # this pdf has 600 above pages that why its taking time

In [7]:
#print(extracted_data[300].page_content)# i have printed the 300th pages

# Load our and create text chunks
*   corpus to text_chunks

In [6]:
def text_split(extracted_data):
    text_spilitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_spilitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)


In [8]:
print("Length of the chunks:",len(text_chunks))

Length of the chunks: 7020


# Embedding

In [7]:
# Now we are going to convert those 7020 chunks data to embedding and store vector DB 

# Download Embedding Model

# Previous Section we used OpenAI embedding model 

# Here we will use HuggingFaceEmbedding
def download_hugging_face_embedding(): # this func download the model huggingface itself
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # Available in HF portal
    return embeddings

embeddings=download_hugging_face_embedding() 
#It got downloaded

In [10]:
embeddings  # This embedding returns the 384 diementions

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query=embeddings.embed_query("My name is karuppas")
print("Length",len(query))  # it return the 384 dienmentions
#Text embeddings

Length 384


In [5]:
import pinecone

pinecone.Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-chat"

host="https://medical-chat-5y3awdy.svc.gcp-starter.pinecone.io"


#docs_search=Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,index_name=index_name)

In [6]:
# https://python.langchain.com/docs/integrations/vectorstores/pinecone

import pinecone

pinecone.Pinecone(api_key=PINECONE_API_KEY)

from langchain_pinecone.vectorstores import Pinecone

vector_db=Pinecone.from_documents(text_chunks,embeddings,index_name=index_name)

In [17]:
vector_db.similarity_search("alergy symtoms",k=2)

[Document(page_content='• inflammation of the mouth ( stomatitis ) or tongue\n(glossitis)\n• insomnia\n• irregular heartbeat\n• loss of appetite\n• nails that are dry, brittle, or ridged\n• rapid breathing\n• sores in the mouth, throat, or rectum\n• sweating\n• swelling of the hands and feet\n• thirst\n• tinnitus (ringing in the ears)\n• unexplained bleeding or bruising\nIn pernicious anemia, the tongue feels unusually\nslick. A patient with pernicious anemia may have:', metadata={'page': 195.0, 'source': 'Data\\Medical_book.pdf'}),
 Document(page_content='usually develop symptoms of deficiency between thesecond and fourth month of life. They are pale, restless,unable to sleep, prone to diarrhea, and have musclewasting and edema in their arms and legs. They have acharacteristic, sometimes silent, cry and develop heartfailure and nerve damage.\nDiagnosis\nA physical examination will reveal many of the', metadata={'page': 483.0, 'source': 'Data\\Medical_book.pdf'})]

In [8]:
vector_db=Pinecone.from_existing_index(index_name,embeddings)

In [9]:
from langchain.llms import OpenAI



In [11]:
retriever=vector_db.as_retriever(search_kwargs={'k':2})

In [1]:
openai_retriever=RetrievalQA.from_chain_type(llm=OpenAI(),chain_type="stuff",retriever=retriever)

NameError: name 'RetrievalQA' is not defined

In [13]:
qa.run("What are allergies")

c:\Users\arung\anaconda3\envs\chatbot\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" Allergies are a response by the body's immune system to a foreign substance, called an allergen, that triggers an abnormal reaction. This reaction can cause symptoms such as sneezing, itching, and swelling."

In [14]:
llm=CTransformers(model="Models/llama-2-7b-chat.ggmlv3.q4_0.bin",
              model_type="llama",
              config={
                  'max_new_tokens':512,
                  'temperature':0.8
              })

In [17]:
ha=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever)

In [18]:
ha.run("What are allergies")

" Allergies are an abnormal reaction to a substance, such as pollen, dust, animal fur, or food. The body's immune system mistakenly identifies the harmless substance as a threat and attempts to fight it off, resulting in an allergic reaction. Common symptoms of allergies include sneezing, runny nose, itchy eyes, and skin rashes. In severe cases, allergies can cause anaphylaxis, a life-threatening reaction that requires immediate medical attention."

In [25]:
ha.run("What are allergies")

" Allergies are an abnormal response to a substance that is typically harmless, and it is often caused by the body's immune system producing antibodies called IgE. These antibodies attach to mast cells, which release histamine and other chemicals into the bloodstream, leading to allergic symptoms such as sneezing, runny nose, and itchy eyes. Common allergens include pollen, dust mites, pet dander, and certain foods like peanuts, fish, and shellfish."

# Prompt

In [19]:


prompt_template=""" 
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:{context}
Question:{question}

Only return the helpfull answer below and nothing else

Helpful answer

"""

In [20]:
PROMPT=PromptTemplate(input_variables=["context","question"],template=prompt_template)
chain_type_kwargs={"prompt":PROMPT}

# Create Question and answering Object
* Refer this website
<a>https://stackoverflow.com/questions/76916636/langchain-chains-not-using-pinecone-vectorstore</a>

* LLM model 
* docs_search which is nothing but its knowledge base where we have all the data n vector format
* other arguments

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [22]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])